In [28]:
# Final project 
# Notes
#1. most important and lengthy part is data preprocessing
#2. not sure why the project asked to vectorize the "text" column and NOT the one which was obtained by removing all stop words, all airline names etc.
#3. when I tried to vectorize that column - called 'preprocessed', the TF-IDF vectorizer started throwing error
#4. Another strange problem that I witnessed was that when I removed punctuations, emojis etc. 
# After that when I started to use "preprocess", the strings started to add backslashes etc. and the counts of top 15 occurences started to distort.
#5. To rectify the above problem, I had to change the code such that I stopped using tokenizing etc. 
#6. Also, to use the Vectorizer there can more than one way a) when we do it for a single text; b) when we do for a column of text taken from a csv file
# Imp Learnings
#1. Using conditional dataframes
#2. removing unnecessary columns by use of "usecols"
#3. many other of course


# All imports

import nltk
from collections import Counter
import docx
import re
from nltk.util import ngrams as ng
from nltk import FreqDist as fd
import pandas as pd
import numpy as np
import docx
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import RocCurveDisplay
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize as wt
from nltk.corpus import stopwords
from nltk.stem import wordnet
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
from sacremoses import MosesDetokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer as twd
import string
string.punctuation
'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [29]:
# Read the training using pandas module and select only the sentiment and text columns 

def df_create(file):
    f = pd.read_csv(file, usecols=["airline_sentiment","text"])
    f.to_csv("final_train_data.csv", index=False)
    
df_create("Final_Project_Tweets-train.csv")

In [30]:
# function to remove the below
# Text contains references with ‘@’ 
#o Text contains links (http , https ) 
#o Text contains punctuations 
#o Text contains Emoticons  

def tokenize(content):
    return wt(content)

def TextAfterRemovingPunctuations(file): 
    df = pd.read_csv(file)
    desc = df['text']
    token = []
    for d in desc:
        tok = tokenize(d)
        token.append(tok)
    to = []
    for t in desc:
        t = str(t)
        #import string
        #string.punctuation
        #'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        no_punct = []
        '''for i in t:
            for j in list(string.punctuation):
                i = i.replace(j,'').lower()
            no_punct.append(i)
        #print(no_punct)'''
        #x = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', t)
        x = re.sub(r'[.,:;@{}()''"!?#&]','', t) # need to figure out a way to handle cleaning of [], all others seem to get handled
        x = re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) # removes https & http
        # Now try removing emoticons
        emoji_pattern =    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
        #print(x)
        text = emoji_pattern.sub(r'', x)# no emoji
        #print(text)
        to.append(text) # this list has all the tokens with punctuation removed and @ removed & emoticons removed
    df['wpehttps'] = to
    df.to_csv(file)

TextAfterRemovingPunctuations("final_train_data.csv")

In [37]:
#removing unnecessary columns keep only 'airline_sentiment', 'wpehttps', 'text'

f = pd.read_csv("final_train_data.csv", usecols=['airline_sentiment', 'wpehttps', 'text'])
f.to_csv("final_train_data.csv", index=False)

In [3]:
# List down the most common 15 words for each sentiment. Observe the results 

def liscommonwordsforeachsentiment(file):
    df = pd.read_csv(file, usecols=['airline_sentiment', 'wpehttps', 'text'])
    #df = pd.read_csv(file)
    #wp = df['removed']
    wp = df['wpehttps']
    #wp = df['wpehttps']
    ai = df['airline_sentiment']
    
    pos_rev = df[df['airline_sentiment'] == "positive"] # conditional dataframes
    neg_rev = df[df['airline_sentiment'] == "negative"]
    neu_rev = df[df['airline_sentiment'] == "neutral"]

    pos = pos_rev['wpehttps']
    po = []
    for w in pos:
        po.append(w)
    po = str(po)
    po = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', po) # need to figure out a way to handle cleaning of [], all others seem to get handled
    #x = re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) # removes https & http
    split_it = po.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Positive Sentiment tweets")
    print(most_occur)

    neg = neg_rev['wpehttps']
    ne = []
    for w in neg:
        ne.append(w)
    ne = str(ne)
    ne = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', ne)
    #ne = re.sub("[.,:;@{}#()''""!?\[|\]]", "", ne)
    split_it = ne.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Negative Sentiment tweets")
    print(most_occur)

    neu = neu_rev['wpehttps']
    nu = []
    for w in neu:
        nu.append(w)
    nu = str(nu)
    nu = re.sub(r"[.,:;@{}#&()''""!?\[|\]]", "", nu)
    #nu = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', nu)
    split_it = nu.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Neutral Sentiment tweets")
    print(most_occur)
    
liscommonwordsforeachsentiment("final_train_data.csv")

<>:46: SyntaxWarning: invalid escape sequence '\['
<>:46: SyntaxWarning: invalid escape sequence '\['
C:\Users\SiyaPransh\AppData\Local\Temp\ipykernel_14548\2695242876.py:46: SyntaxWarning: invalid escape sequence '\['
  nu = re.sub(r"[.,:;@{}#&()''""!?\[|\]]", "", nu)


Most occuring 15 words from Positive Sentiment tweets
[('to', 668), ('the', 658), ('for', 486), ('you', 457), ('I', 417), ('a', 352), ("'SouthwestAir", 319), ("'JetBlue", 283), ('and', 281), ("'united", 274), ('my', 243), ('in', 229), ('on', 220), ('flight', 211), ('thanks', 190)]
Most occuring 15 words from Negative Sentiment tweets
[('to', 4284), ('the', 2900), ('I', 2446), ('a', 2243), ('for', 1971), ('and', 1951), ('on', 1920), ('my', 1593), ('you', 1557), ('flight', 1554), ('is', 1431), ('in', 1216), ("'united", 1163), ('of', 1125), ("'USAirways", 1059)]
Most occuring 15 words from Neutral Sentiment tweets
[('to', 1173), ('I', 795), ('the', 697), ('a', 577), ('you', 497), ('on', 455), ('for', 433), ('SouthwestAir', 381), ('united', 370), ('my', 361), ('JetBlue', 349), ('flight', 339), ('in', 337), ('and', 323), ('is', 311)]


In [38]:
# Remove Stopwords from all the tweets. Save changes in a new column and list down most common 15 words. 
# without tokenizing

def RemoveStopWords(sentence):
    filtered_sentence = []
    for w in sentence:
        if w.lower() not in stop_words:
            filtered_sentence.append(w)
        #print(filtered_sentence)
    return (filtered_sentence)

def tokenize(content):
    return wt(content)

def rsw(file):
    df = pd.read_csv(file, usecols=['airline_sentiment', 'text', 'wpehttps'])
    wp = df['wpehttps']
    ai = df['airline_sentiment']
    content = []
    c = []
    #print(type(wp))
    for w in wp:
        #print(type(w))
        #print(w)
        tt = w
        content.append(tt)
    sentences = []
    for we in content:
        sentence = we
        for word in stop_words:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
            sentence = sentence.replace("  ", " ")
            sentence = sentence.replace("https", " ")
            sentence = sentence.replace("https", " ")
        sentences.append(sentence)
    df['rsw'] = sentences
    df.to_csv(file)

rsw("final_train_data.csv")

In [39]:
#removing unnecessary columns keep only 'airline_sentiment', 'wpehttps', 'text'

f = pd.read_csv("final_train_data.csv", usecols=['airline_sentiment', 'wpehttps', 'text', 'rsw'])
f.to_csv("final_train_data.csv", index=False)

In [5]:
# List down the most common 15 words for each sentiment. Observe the results 

def liscommonwordsforeachsentiment(file):
    df = pd.read_csv(file, usecols=['airline_sentiment', 'wpehttps', 'text', 'rsw'])
    #df = pd.read_csv(file)
    #wp = df['removed']
    wp = df['rsw']
    #wp = df['wpehttps']
    ai = df['airline_sentiment']
    
    pos_rev = df[df['airline_sentiment'] == "positive"] # conditional dataframes
    neg_rev = df[df['airline_sentiment'] == "negative"]
    neu_rev = df[df['airline_sentiment'] == "neutral"]

    pos = pos_rev['rsw']
    po = []
    for w in pos:
        po.append(w)
    po = str(po)
    po = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', po) # need to figure out a way to handle cleaning of [], all others seem to get handled
    #x = re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) # removes https & http
    split_it = po.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Positive Sentiment tweets")
    print(most_occur)

    neg = neg_rev['rsw']
    ne = []
    for w in neg:
        ne.append(w)
    ne = str(ne)
    ne = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', ne)
    #ne = re.sub("[.,:;@{}#()''""!?\[|\]]", "", ne)
    split_it = ne.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Negative Sentiment tweets")
    print(most_occur)

    neu = neu_rev['rsw']
    nu = []
    for w in neu:
        nu.append(w)
    nu = str(nu)
    nu = re.sub(r"[.,:;@{}#&()''""!?\[|\]]", "", nu)
    #nu = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', nu)
    split_it = nu.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Neutral Sentiment tweets")
    print(most_occur)
    
liscommonwordsforeachsentiment("final_train_data.csv")

Most occuring 15 words from Positive Sentiment tweets
[('I', 417), ("'SouthwestAir", 342), ("'JetBlue", 304), ("'united", 288), ('flight', 211), ("'AmericanAir", 195), ('thanks', 190), ('Thank', 169), ("'USAirways", 161), ('Thanks', 150), ('thank', 144), ("',", 124), ('great', 111), ('service', 97), ("you',", 94)]
Most occuring 15 words from Negative Sentiment tweets
[('I', 2446), ('flight', 1554), ("'united", 1407), ("'USAirways", 1270), ("'AmericanAir", 1062), ('get', 691), ("'SouthwestAir", 655), ('Cancelled', 643), ("'JetBlue", 480), ('hours', 448), ('service', 435), ('Flight', 398), ('"united', 396), ('hold', 394), ('2', 391)]
Most occuring 15 words from Neutral Sentiment tweets
[('I', 795), ('SouthwestAir', 410), ('united', 391), ('JetBlue', 372), ('flight', 341), ('AmericanAir', 304), ('USAirways', 250), ('get', 171), ('-', 134), ('flights', 125), ('VirginAmerica', 115), ('help', 103), ('please', 98), ('Im', 97), ('need', 96)]


<>:46: SyntaxWarning: invalid escape sequence '\['
<>:46: SyntaxWarning: invalid escape sequence '\['
C:\Users\SiyaPransh\AppData\Local\Temp\ipykernel_28696\2846197693.py:46: SyntaxWarning: invalid escape sequence '\['
  nu = re.sub(r"[.,:;@{}#&()''""!?\[|\]]", "", nu)


In [40]:
#removing unnecessary columns keep only 'airline_sentiment', 'wpehttps', 'text'

f = pd.read_csv("final_train_data.csv", usecols=['airline_sentiment', 'wpehttps', 'text', 'rsw'])
f.to_csv("final_train_data.csv", index=False)

In [41]:
# Remove ('americanair', 'united', 'delta', 'southwestair', 'jetblue', 'virginamerica', 'usairways', 'flight', 'plane' ) from all the tweets. Save changes in a new column and list down most common 15 words. 

def RemoveStopWords(sentence):
    filtered_sentence = []
    for w in sentence:
        if w.lower() not in stop_words:
            filtered_sentence.append(w)
        #print(filtered_sentence)
    return (filtered_sentence)

def tokenize(content):
    return wt(content)

def rsw(file):
    df = pd.read_csv(file, usecols=['airline_sentiment', 'text', 'wpehttps','rsw'])
    wp = df['wpehttps']
    ai = df['airline_sentiment']
    content = []
    c = []
    list_words = ['americanair', 'united', 'delta', 'southwestair', 'jetblue', 'virginamerica', 'usairways', 'flight', 'plane', 'the', 'i']
    #print(type(wp))
    for w in wp:
        #print(type(w))
        #print(w)
        tt = w
        content.append(tt)
    sentences = []
    for we in content:
        sentence = we.lower()
        for word in list_words:
            word = word.lower()
            token = " " + word + " "
            token1 = word + " "
            token2 = " " + word
            sentence = sentence.replace(token, " ")
            sentence = sentence.replace(token1, " ")
            sentence = sentence.replace(token2, " ")
            sentence = sentence.replace("  ", " ")
        sentences.append(sentence)
    df['removed'] = sentences
    df.to_csv(file)

rsw("final_train_data.csv")

In [26]:
# List down the most common 15 words for each sentiment. Observe the results 

def liscommonwordsforeachsentiment(file):
    df = pd.read_csv(file, usecols=['airline_sentiment', 'wpehttps', 'text', 'rsw','removed'])
    #df = pd.read_csv(file)
    #wp = df['removed']
    wp = df['removed']
    #wp = df['wpehttps']
    ai = df['airline_sentiment']
    
    pos_rev = df[df['airline_sentiment'] == "positive"] # conditional dataframes
    neg_rev = df[df['airline_sentiment'] == "negative"]
    neu_rev = df[df['airline_sentiment'] == "neutral"]

    pos = pos_rev['removed']
    po = []
    for w in pos:
        po.append(w)
    po = str(po)
    po = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', po) # need to figure out a way to handle cleaning of [], all others seem to get handled
    #x = re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) # removes https & http
    split_it = po.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Positive Sentiment tweets")
    print(most_occur)

    neg = neg_rev['removed']
    ne = []
    for w in neg:
        ne.append(w)
    ne = str(ne)
    ne = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', ne)
    #ne = re.sub("[.,:;@{}#()''""!?\[|\]]", "", ne)
    split_it = ne.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Negative Sentiment tweets")
    print(most_occur)

    neu = neu_rev['removed']
    nu = []
    for w in neu:
        nu.append(w)
    nu = str(nu)
    nu = re.sub(r"[.,:;@{}#&()''""!?\[|\]]", "", nu)
    #nu = re.sub(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~','', nu)
    split_it = nu.split()
    Counters_found = Counter(split_it)
    most_occur = Counters_found.most_common(15)
    print("Most occuring 15 words from Neutral Sentiment tweets")
    print(most_occur)
    
liscommonwordsforeachsentiment("final_train_data.csv")

Most occuring 15 words from Positive Sentiment tweets
[("'", 1316), ('the', 689), ('to', 675), ('you', 529), ('for', 492), ('i', 441), ('a', 369), ('thanks', 341), ('thank', 327), ('"', 325), ('and', 304), ('my', 262), ('in', 235), ('on', 232), ('your', 186)]
Most occuring 15 words from Negative Sentiment tweets
[('to', 4304), ("'", 4231), ('the', 3008), ('i', 2600), ('"', 2339), ('a', 2296), ('and', 2037), ('for', 1988), ('on', 1971), ('you', 1750), ('my', 1716), ('is', 1506), ('in', 1248), ('of', 1134), ('your', 986)]
Most occuring 15 words from Neutral Sentiment tweets
[('to', 1182), ('i', 842), ('the', 726), ('a', 588), ('"', 521), ('you', 518), ('on', 468), ('for', 441), ('my', 386), ('is', 371), ('in', 346), ('and', 328), ('can', 296), ('it', 248), ('of', 245)]


<>:46: SyntaxWarning: invalid escape sequence '\['
<>:46: SyntaxWarning: invalid escape sequence '\['
C:\Users\SiyaPransh\AppData\Local\Temp\ipykernel_28696\1250503209.py:46: SyntaxWarning: invalid escape sequence '\['
  nu = re.sub(r"[.,:;@{}#&()''""!?\[|\]]", "", nu)


In [42]:
#removing unnecessary columns keep only 'airline_sentiment', 'wpehttps', 'text', 'rsw','removed'

f = pd.read_csv("final_train_data.csv", usecols=['airline_sentiment', 'wpehttps', 'text', 'rsw','removed'])
f.to_csv("final_train_data.csv", index=False)

In [45]:
# Encode Sentiments using Label Encoder 

def label_encode(file):
    df = pd.read_csv(file)
    label_encoder = LabelEncoder()
    c = df['airline_sentiment']
    encoded_data = label_encoder.fit_transform(c)
    df['label'] = encoded_data
    df.to_csv(file)
    
label_encode("final_train_data.csv")

In [56]:
#removing unnecessary columns keep only 'airline_sentiment', 'wpehttps', 'text', 'rsw','removed'

f = pd.read_csv("final_train_data.csv", usecols=['airline_sentiment', 'wpehttps', 'text', 'rsw','removed', 'label'])
f.to_csv("final_train_data.csv", index=False)

In [57]:
# Vectorize the Text Column. 

def tokenize(content):
    return wt(content)

def RemoveStopWords(content):
    filtered_sentence = []
    for w in content:
        if w.lower() not in stop_words:
            filtered_sentence.append(w)
    return (filtered_sentence)

def Lemmatize(content):
        return(wl.lemmatize(content))

def refine(user_string):     
    #val = input("Enter your string: ")     
    #print("Entered String : ", str(user_string))     
    text_tokens = tokenize(user_string)     
    #print("Entered String (tokenized):\n", text_tokens)     
    removed_stop_words = text_tokens 
    #print("Entered String (Stop Words Removed):\n", removed_stop_words)     
    #lemma =  Lemmatize(list(removed_stop_words))     
    #print("Entered String (Lemmatized):\n") 
    lemmatized_list = []
    for word in removed_stop_words:         
        w = Lemmatize(word) 
        w = str(w)
        if (w != ""):
            lemmatized_list.append(w)
    return(lemmatized_list) 

def AddRefinedDescription(file):
    df = pd.read_csv(file)
    #print(df)
    desc = df['text']
    #print(desc)
    refined_d = []
    for d in desc:
        refined = refine(d)
        refined_d.append(refined)
    df['preprocessed'] = refined_d
    df.to_csv("final_train_data.csv")
    #print(df)

def TFIDFVectorization(S1):
    tf_vect = TfidfVectorizer(min_df=1,lowercase=True,stop_words='english')
    review_list = [S1]
    tf_matrix = tf_vect.fit_transform(review_list)
    #print(tf_matrix.toarray())
    return(tf_matrix.toarray())

def CreateTFIDFVectorization(file):
    df = pd.read_csv(file)
    desc = df['preprocessed']
    refined_tokens = []
    for d in desc:
        #print(d)
        refined = TFIDFVectorization(d)
        refined_tokens.append(refined)
    df['TF-IDF Vectorizer'] = refined_tokens
    df.to_csv(file)

AddRefinedDescription("final_train_data.csv")
CreateTFIDFVectorization("final_train_data.csv")

In [58]:
#removing unnecessary columns keep only 'airline_sentiment', 'wpehttps', 'text', 'rsw','removed'

f = pd.read_csv("final_train_data.csv", usecols=['airline_sentiment', 'wpehttps', 'text', 'rsw','removed', 'label', 'TF-IDF Vectorizer', 'preprocessed'])
f.to_csv("final_train_data.csv", index=False)

In [59]:
def cm(file):
    df = pd.read_csv(file)
    desc = df['preprocessed']
    y = df['label']
    count_vect = TfidfVectorizer(min_df=1,lowercase=True,stop_words='english')
    X_counts = count_vect.fit_transform(desc)
    #print(type(X_counts))
    #print(X_counts.shape)
    X_names = count_vect.get_feature_names_out()
    X_counts = pd.DataFrame(X_counts.toarray(), columns=X_names)
    #print(X_counts.shape)
    #print(X_counts.head())
    X_counts.to_csv("data_final.csv", sep=',', encoding='utf-8', index=False, header=True)
    y.to_csv("data_final.csv")
    
cm("final_train_data.csv")

<class 'scipy.sparse._csr.csr_matrix'>
(10640, 11753)
(10640, 11753)
    00  000  000114  000lbs  0016  00a  00am  00p  00pm   01  ...  zrh  \
0  0.0  0.0     0.0     0.0   0.0  0.0   0.0  0.0   0.0  0.0  ...  0.0   
1  0.0  0.0     0.0     0.0   0.0  0.0   0.0  0.0   0.0  0.0  ...  0.0   
2  0.0  0.0     0.0     0.0   0.0  0.0   0.0  0.0   0.0  0.0  ...  0.0   
3  0.0  0.0     0.0     0.0   0.0  0.0   0.0  0.0   0.0  0.0  ...  0.0   
4  0.0  0.0     0.0     0.0   0.0  0.0   0.0  0.0   0.0  0.0  ...  0.0   

   zrh_airport  zsdgzydnde  zsuztnaijq  ztrdwv0n4l  zukes  zurich  zv2pt6trk9  \
0          0.0         0.0         0.0         0.0    0.0     0.0         0.0   
1          0.0         0.0         0.0         0.0    0.0     0.0         0.0   
2          0.0         0.0         0.0         0.0    0.0     0.0         0.0   
3          0.0         0.0         0.0         0.0    0.0     0.0         0.0   
4          0.0         0.0         0.0         0.0    0.0     0.0         0.0   


In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import RocCurveDisplay

def run_training(file):
    #train, test = train_test_split(df, test_size=0.2)
    df = pd.read_csv(file)
    dfy = df['label']
    dfx = df.drop('label', axis=1)
    #print(dfx.shape)
    #print(dfy.shape)
    
    X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.2)
    #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    c = MultinomialNB() 
    c.fit(X_train, y_train)
    y_pred = c.predict(X_test)
    #print(type(y_pred))

    actualValue = y_test
    predictedValue = y_pred
    print("Confusion Matrix :")
    cmt = confusion_matrix(actualValue, predictedValue)
    #tn, fp, fn, tp = confusion_matrix(actualValue, predictedValue).ravel()
    print(cmt)
    print("Accuracy :")
    acc = accuracy_score(actualValue, predictedValue)
    print(acc)

run_training("data_final.csv")

Confusion Matrix :
[[1325    0    0]
 [ 436    0    0]
 [ 367    0    0]]
Accuracy :
0.6226503759398496
